<a href="https://colab.research.google.com/github/maitreya-v/Transfer-Learning-DistilBERT/blob/main/Distillbert_Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import numpy as np

In [37]:
df=pd.read_csv('/content/SMSCollection.csv',names=['label','message'])

In [38]:
df.head()

,label,message
0,Class,sms
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...


In [39]:
df.drop(0,inplace=True)

In [40]:
df = df.reset_index(drop=True)

In [41]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [42]:
df.shape

(5572, 2)

In [43]:
X=list(df['message'])
y=list(df['label'])

In [44]:
y=np.array(pd.get_dummies(y,drop_first=True))

In [45]:
type(y)

numpy.ndarray

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [47]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [48]:
X_train[0]

"No I'm good for the movie, is it ok if I leave in an hourish?"

In [50]:
train_encodings = tokenizer(X_train,truncation=True,padding=True)
test_encodings = tokenizer(X_test,truncation=True,padding=True)

In [59]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [54]:
import tensorflow as tf

train_dataset=tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset=tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [60]:
type(train_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [63]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)